In [1]:
from astropy.io import ascii
import glob
import matplotlib.pyplot as plt
import numpy as np
from scipy import asarray as ar, exp
from scipy.optimize import curve_fit, least_squares
import re
%matplotlib inline

snls_colors = {"sdssg":"darkgreen", "sdssi":"darkblue", "sdssr":"darkred", "sdssz":"purple"}
base_colors = {"sdssg":"green", "sdssi":"blue", "sdssr":"red", "sdssz":"magenta"}

import astropy.coordinates as coord
import astropy.units as u
import Utils.DiaSourceTools as DSTools

import multiprocessing

import sncosmo

In [2]:
def read_lc_files(obj):
    table = ascii.read(obj)
    return table

In [3]:
f = open('/renoir_data_02/jpreyes/lsst_data/sn_control/J_A+A_523_A7_table9.dat.txt','r')
data_elems = f.read()
#elems = re.findall('^(.*?D3.*?)\\|(.*?)\\|', data_elems, re.MULTILINE)
elems = re.findall('^(05D3.*?)\\|(.*?)\\|(.*?)\\|(.*?)\\|(.*?)\\|(.*?)\\|(.*?)\\|', data_elems, re.MULTILINE)
f.close()

f = open('/renoir_data_02/jpreyes/lsst_data/sn_control/J_A+A_523_A7_table10.dat.txt','r')
data = f.read()
f.close()

In [4]:
snls_array = []
for sn in elems:

        c = coord.SkyCoord(sn[1], unit=(u.hourangle, u.deg))

        m = re.findall('^'+str(sn[0])+'\\|(r|g|z|i)\\|(.*?)\\|(.*?)\\|(.*?)$', data, re.MULTILINE)

        snls_lc = DSTools.build_light_curve_from_snls_file_2(m, c, sn[0], sn[-1])

        if len(m)>0:
            #print sn[0], c.ra.deg, c.dec.deg

            #plt.errorbar(snls_lc['mjd'], snls_lc['flux'], yerr=snls_lc['flux_error'], fmt='.', color='blue')
            #show_multi_filter_light_curve(snls_lc, snls_colors)
            plt.show()
            snls_array.append(snls_lc)

print("total of sn:" + str(len(snls_array)))

total of sn:33


In [5]:
def match_with_lc(slc):
    matches = []
    for lc in snls_array:
        #print "light curve"
        val = {"ra":lc[0]["ra"], "dec":lc[0]["dec"]}
        #print np.rad2deg(lc[0]["ra"]), np.rad2deg(lc[0]["dec"])
        

        comp = {"ra":np.mean(slc["ra"]), "dec":np.mean(slc["dec"])}
        if DSTools.source_distance(val, comp)<1:
            #print "match"
            matches.append((lc,slc))


    #print len(matches)
    return matches


In [97]:
def get_total_matches_all(sigma):
    lienObj = glob.glob("/datadec/cppm/jpreyes/CFHT_Production/light_curves_corrected_"+str(float(sigma))+"sigma/*.dat")



    try:
        pool = multiprocessing.Pool()
        res =  pool.map(read_lc_files, lienObj)
    finally:
        pool.close()

    s_candidates = res
    print len(s_candidates)

    try:
        pool = multiprocessing.Pool()
        match_res =  pool.map(match_with_lc, s_candidates)
    finally:
        pool.close()

    res_final = []
    for r in match_res:
        res_final +=r

    count = 0
    ids = []
    for control, match in res_final:
        if control[0]["id"] not in ids:
            count+=1
            ids.append(control[0]["id"])

    print count
    
    return s_candidates, res_final

def get_total_matches(f, sigma):
    lienObj = glob.glob("/datadec/cppm/jpreyes/CFHT_Production/light_curves_corrected_"+f+"_"+str(float(sigma))+"sigma/*.dat")



    try:
        pool = multiprocessing.Pool()
        res =  pool.map(read_lc_files, lienObj)
    finally:
        pool.close()

    s_candidates = res
    print len(s_candidates)

    try:
        pool = multiprocessing.Pool()
        match_res =  pool.map(match_with_lc, s_candidates)
    finally:
        pool.close()

    res_final = []
    for r in match_res:
        res_final +=r

    count = 0
    ids = []
    for control, match in res_final:
        if control[0]["id"] not in ids:
            count+=1
            ids.append(control[0]["id"])

    print count
    
    return s_candidates, res_final

In [23]:
def count_without_duplicates(res_final):
    count = 0
    ids = []
    for control, match in res_final:
        if control[0]["id"] not in ids:
            count+=1
            ids.append(control[0]["id"])

    return count


In [65]:
c, rf = get_total_matches("i",20)

50791
24


In [66]:
len(c)

50791

In [67]:
selected_candidates = []
for candidate in c:
    positives = len(candidate[candidate["classification"]==6])+len(candidate[candidate["classification"]==7])
    total = len(candidate)
    if float(positives)/float(total) > 0.5:
        selected_candidates.append(candidate)
print len(selected_candidates)

47230


In [68]:
try:
    pool = multiprocessing.Pool()
    match_res =  pool.map(match_with_lc, selected_candidates)
finally:
    pool.close()

ms = []
for r in match_res:
    ms +=r


In [69]:
len(ms)
print(count_without_duplicates(ms))

23


In [30]:
selected_candidates[0][selected_candidates[0]["classification"]==6]

mjd,classification,zpsys,flux,ra,flux_error,dec,zp,bandpass,magnitude,magnitude_error
float64,float64,str2,float64,float64,float64,float64,float64,str5,float64,float64
53390.0,6.0,ab,20.904378891,3.7641558018,1.52051723003,0.913615416744,30.0,sdssr,26.6994068291,0.0789729565856
53473.0,6.0,ab,26.4316043854,3.76415595799,1.80007195473,0.913615412181,30.0,sdssr,26.4446911865,0.0739419092357
53469.0,6.0,ab,27.6053237915,3.76415602445,1.25423407555,0.913615439759,30.0,sdssr,26.3975178863,0.0493298812703
53461.0,6.0,ab,22.8709926605,3.76415733798,2.17728543282,0.913615333629,30.0,sdssr,26.6017874637,0.10336051686
53466.0,6.0,ab,23.8397731781,3.7641572501,1.6307643652,0.913615349991,30.0,sdssr,26.5567447024,0.0742699982712
53446.0,6.0,ab,26.4294548035,3.76415692869,1.32050311565,0.913615336405,30.0,sdssr,26.4447794889,0.0542469775416
53527.0,6.0,ab,29.6612930298,3.76415606165,1.10508298874,0.913615317759,30.0,sdssr,26.3195248016,0.0404509880581
53531.0,6.0,ab,25.6310558319,3.76415711426,1.12678265572,0.913615291352,30.0,sdssr,26.4780837584,0.0477307190241
53505.0,6.0,ab,22.5271110535,3.76415583545,1.21694970131,0.913615414764,30.0,sdssr,26.6182362499,0.058653164489


In [65]:
get_total_matches("i",5)
get_total_matches("i",10)
get_total_matches("i",15)
get_total_matches("i",20)
get_total_matches("i",25)

57413
24
55989
24
54009
24
50791
24
45707
24


In [68]:
get_total_matches("g",5)
get_total_matches("g",10)
get_total_matches("g",15)
get_total_matches("g",20)
get_total_matches("g",25)

18197
10
17917
10
17605
10
17270
10
16789
10


In [ ]:
get_total_matches("r",0)
get_total_matches("i",0)
get_total_matches("g",0)



46382


In [96]:
get_total_matches_all(5)
get_total_matches_all(10)
get_total_matches_all(15)
get_total_matches_all(20)
get_total_matches_all(25)

0
0
0
0
0
0
0
0
0
0


([], [])